# Project 2: Policy Change
## Quantified Cognition
### Psychology 5332


# Name: *Your Name Here*
# User ID: *Your ID Here*

# Objectives

Upon completion of this assignment, the student will demonstrate the ability to:

1. Modify the policy of an RL model
2. Test whether the new policy improves learning in various environments
3. Evaluate why the policy did or did not work and propose alternative approaches


# Assignment

- The goal of this assignment is to modify the *policy* of the successor representation model to generate different behaviors given the environment. You will then test whether this change improves learning in different environmental contexts. 

- You will perform this assignment by writing code in *this notebook* (***after making a copy and renaming it to have your userid in the title --- e.g., P02_Policy_Change_mst3k***).

- ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVACollab.***

## HINTS

- Be sure to comment your code
- I have provided cells with general instructions for what they should contain.
  

# Policy Change

As discussed in class, both learning and behavior in an environment depend on the agent's policy for selecting actions given the current state. We also discussed the importance of the exploration vs. exploitation trade-off during learning. Whether you exploit currently preferred actions or explore novel actions also depends on the policy.

Here, we will modify the policy of the successor representation (SR) reinforcement learning (RL) model we covered in class. Specifically, the current policy involves evaluating future visitions to other states given a current state and all possible actions, and calculating a Q value for each action taking into account the learned rewards (or punishments) at each state. Mathematically, that involves:

$$Q = (M \cdot s_i) \cdot r,$$

where $M$ is the successor representation matrix, $s_i$ is the current state, and $r$ is the vector of rewards associated with each state.

What we'd like to do is encourage the agent to move away from their most recent states, thereby enhancing exploration of the environment. Luckily, the agent already keeps track of recent states in the eligibility trace (or context, in the parlance of the temporal context model). Your job is to use this vector (which is already passed into the policy function as `t0`) along with a new parameter (let's call this `revisit_penalty`) that can take on values between zero and one, to modify the rewards associated with each state such that recent states are penalized.

Once you have updated the policy code, train the model in an environment that is not slippery (i.e., set `slippery = False`) and make note of how many training iterations it needs to learn to a criterion of 25 correct in a row. Does the model learn faster when you penalize more for revisiting recent states? Make sure to assess this in a handful of random environments by rerunning the code that generates a random map (see below), so that your assessment isn't biased by a single environment.

Next, make the environment slippery (i.e., set `slippery = True`) and see if penalizing revisitation of recent states helps or hurts learning. Is the model able to learn better or worse on average? Even if it never reaches the criterion, is it solvinging more or less often with the penalty?

In addition to the questions above, here are some questions to answer in your write-up:

- Why do you think you observed what you did?
- Can you think of a different policy that may work better in slippery environments?


In [ ]:
# to install more libraries
!pip install gymnasium

In [1]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import matplotlib.pyplot as plt   # plotting

from IPython.display import display, clear_output, Image
import time

from fastprogress.fastprogress import master_bar, progress_bar

#import gym
#from gym.envs.toy_text import frozen_lake

import gymnasium as gym
from gymnasium.envs.toy_text import frozen_lake

## Environment defined from class

In [ ]:
# define the environment
size = 10
p_frozen = 0.9
slippery = False

# generate a random map
desc = frozen_lake.generate_random_map(size=size, p=p_frozen)
env = frozen_lake.FrozenLakeEnv(desc=desc,
                                is_slippery=slippery,
                                render_mode='ansi'
                               )

## reset the environment and get the initial state
observation, info = env.reset()
display(print(env.render()))

## Policy and Model from class

In [ ]:
# params
gamma = .95
alpha = .5
rho = .25
tau = 10.0
p_rand = 0.0
hole_penalty = -1.0
off_board_penalty = -0.0

# set up our agent
# pull out the number of actions and unique states
n_actions = env.action_space.n
n_states = env.observation_space.n

# create orthogonal state representations
states = np.eye(n_states)

# allocate for where we learn:
# rewards associated with each state
rewards = np.zeros(n_states)
# states associated with each other (via SR)
M = np.zeros((n_actions, n_states, n_states))

# keep track of scores during learning
scores = []

# define a policy 
# !!!!! MODIFY THIS CODE TO UPDATE THE POLICY !!!!
def pick_action(f0, t0, M, rewards, tau, p_rand=0.0):
    # apply policy to pick action
    if p_rand > 0.0 and np.random.rand() < p_rand:
        # pick a random action
        action = env.action_space.sample()
    else:
        Q = np.dot(np.dot(M, f0), rewards)
        #action = np.random.choice(np.where(Q==Q.max())[0])
        #action = np.argmax(Q)
        pQ = np.exp(Q*tau)/np.exp(Q*tau).sum()
        action = np.argmax(np.random.rand() < np.cumsum(pQ))
    return action


In [ ]:
ntrials = 1000
max_moves = 500
max_corr = 25

for r in progress_bar(range(ntrials)):
    # reset for new attempt at recovering the frisbee
    observation, info = env.reset()
    last_obs = observation
    f0 = states[observation]
    t0 = states[observation]
    
    # set current annealing
    cur_p_rand = p_rand*(1-((r+1)/ntrials))

    for i in range(max_moves):
        # pick an action
        action = pick_action(f0, t0, M, rewards, tau, p_rand=cur_p_rand)  
    
        # observe new state
        observation, reward, trunc, done, info = env.step(action)
        
        # turn the new state into a vector representation
        f1 = states[observation]

        # learn via successor representation
        # prediction from previous state
        p0 = np.dot(M[action], f0)
        
        # observed outcome, plus discounted future prediction
        # when following that policy
        f1_action = pick_action(f1, t0, M, rewards, tau, p_rand=cur_p_rand)
        o1 = (f1 + gamma*(np.dot(M[f1_action], f1)))
        
        # update the association for that action
        M[action] += alpha * np.outer((o1 - p0), t0)

        # update context (eligibility trace)
        #t1 = rho*t0 + (1-rho)*f1
        t1 = np.clip(rho*t0 + f1, 0, 1)

        # process the reward if any
        if trunc and reward==0:
            # get negative rewards for falling in a hole
            reward = hole_penalty
            
        if last_obs == observation:
            # action gave rise to no change in movement
            reward = off_board_penalty
            
        #if reward == 0:
        #    # punish going to a state and not getting anything for it
        #    rewards[last_obs] -= .1

        # update our representation of rewards/punishments at the observed state
        rewards[observation] += alpha*(reward - rewards[observation])

        # see if we're done
        if trunc:
            #print("Episode finished after {} timesteps with reward {}".format(i+1, reward))
            # save out our final reward/punishment
            scores.append(reward)
            break

        # prepare for next iteration
        f0 = f1
        t0 = t1
        last_obs = observation
    
    # if we ran out of time, say we fell in
    if i==(max_moves-1):
        scores.append(hole_penalty)
        
    if len(scores)>max_corr and np.mean(scores[-max_corr:])==1.0:
        # we're consistently solving it, so quit
        break

# render the final state
env.render()

# plot a moving average of scores
N=10
plt.plot(np.convolve(scores, np.ones((N,))/N, mode='valid'))

print("Mean final performance:", np.mean(scores[-max_corr:]))

### Write your short answer here to the questions listed above:
